In [1]:
import torch

# Local imports
from train import train_model

/scratch/local3/2024/aguo/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")

print(f"Using device: {device}")

Using device: cuda:0


In [3]:
args = {
    "use_small_dataset": False,
    "batch_size": 64,
    "stoppage_epochs": 32,
    "max_epochs": 512,
    "seed": 0,
    "data_path": "../data",
    "protein_graph_dir": "../data/protein_graphs",
    "frac_train": 0.8,
    "frac_validation": 0.1,
    "frac_test": 0.1,
    "huber_beta": 0.5,
    "weight_decay": 5e-4,
    "lr": 1e-4,
    "scheduler_patience": 10,
    "scheduler_factor": 0.5,
    "hidden_size": 128,
    "emb_size": 128,
    "num_layers": 4,
    "num_attn_heads": 8,
    "dropout": 0.2,
    "mlp_dropout": 0.4,
    "pooling_dim": 128,
    "mlp_hidden": 192,
    "max_nodes": 80, # Max number of amino acids
    "model_path": '../models/model1.pth'
}

In [ ]:
import argparse

training_args = argparse.Namespace(**args)
train_model(training_args, device)

Model parameters: 2066125


Epoch 1/512: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 906/906 [03:15<00:00,  4.63it/s]


Epoch 1/512: Train Loss=0.98560, MSE=2.74826, MAE=1.21228, Acc=0.52080 | Val Loss=0.66896, MSE=1.21675, MAE=0.89025, Acc=0.62350


Epoch 2/512: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 906/906 [03:02<00:00,  4.96it/s]


Epoch 2/512: Train Loss=0.75729, MSE=1.48649, MAE=0.98104, Acc=0.57668 | Val Loss=0.67169, MSE=1.22355, MAE=0.89311, Acc=0.62212


...

Epoch 29/512: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 906/906 [02:57<00:00,  5.10it/s]


Epoch 29/512: Train Loss=0.54050, MSE=0.92744, MAE=0.75456, Acc=0.71683 | Val Loss=0.46180, MSE=0.76444, MAE=0.66925, Acc=0.77178


Epoch 30/512: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 906/906 [03:11<00:00,  4.73it/s]


Epoch 30/512: Train Loss=0.53393, MSE=0.91436, MAE=0.74725, Acc=0.72095 | Val Loss=0.44546, MSE=0.72825, MAE=0.65186, Acc=0.78255


Epoch 31/512:  92%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                     | 832/906 [03:33<00:28,  2.60it/s]